## 7.4 维度属性层次
虽然本章所描述的预算链用例相当简单，但它包含了一系列层次，以及需呀设计者考虑的一系列选择。至少存在4种层次：日历层次、账户层次、地理层次和组织层次  
### 7.4.1 固定深度的位置层次
在预算链中，日历层次是场景的深度固定的层次，每个层次都是具有实际意义的标记  
账户维度也可以存在固定的多对一层次，例如，总经理层、业务主管层和业务经理层账户  
在固定位置维度中，重要的每个层次具有特定的名称  
注意：避免在固定位置层次中使用抽象的名称，例如，级别-1，级别-2等等 5  
### 7.4.2 具有轻微不整齐的可变深度层次  
地理层次是一种具有挑战性的层次。如图7-7所示，存在三种可能性
![7-7](../syn_pic/the_datawarehouse_toolkit/7-7.png)
<center>图7-7 同时存在于单个位置维度中包含简单、中等和复杂层次的示例数据值</center>

### 7.4.3 不整齐可变深度层次
在预算用例中，组织结构是不整齐可变深度层次的最好示例。设想某个企业包含如图7-8所示的具有13个组织的上卷结构。此类组织都有自己的预算、委托和支付  
![7-8](../syn_pic/the_datawarehouse_toolkit/7-8.png)
<center>图7-8 组织上卷结构</center>

对某个组织，可以通过简单连接组织维度和事实表，查询某个特定账户的预算。如图7-9所示。也可能希望针对数的一部分或整棵树上卷预算 5  
![7-9](../syn_pic/the_datawarehouse_toolkit/7-9.png)
<center>图7-9 连接事实表的组织维度</center>

表示父/子树结构的典型方法是在组织维度的每行中设置指向其父节点的递归指针，如图7-10所示  
![7-10](../syn_pic/the_datawarehouse_toolkit/7-10.png)
<center>图7-10 典型的父/子递归设计</center>

从一个上卷结构转换到另外一个上卷结构是无法实现的，因为多数递归指针都被破坏性地修改。要维护组织中的类型2缓慢变化维度属性也是不可能的，因为对高层节点关键字的改变需要对整棵树的关键字进行修改  
解决表示任意上卷结构问题的方法是建立特殊的与朱维度表无关的，包含与上卷有关的所有信息的桥接表。该桥接表的粒度是树中每个从父节点到该父节点下的所有子节点的路径，如图7-11所示 5  
 ![7-11](../syn_pic/the_datawarehouse_toolkit/7-11.png)
<center>图7-11 组织映射桥接表样例行</center>

图7-8所描述的样例树产生了图7-11所示的43行  
映射表中最顶层父节点标识表示发自顶层父节点的特定路径。最低层子节点标识树的“叶节点”特定路径  
如果对组织维度表建立单行约束，可以将维度表与映射表和组织表连接，如图7-12所示  
 ![7-12](../syn_pic/the_datawarehouse_toolkit/7-12.png)
<center>图7-12 将组织映射桥接表连接到事实表 5</center>

注意：文章“Building Hierarchy Bridge Tables”提供了本节所描述的建立层次桥接表的代码示例  
但使用隐射桥接表约束组织维度到单一行时必须非常小心，否则可能会冒重复计算树中子节点和孙子节点的风险  
在此情况下，需要建立一个自定义查询，而不是简单地进行连接，具体采用下列约束：  
### 7.4.4 不规则层次中的共享所有权  
映射表可以表示部分所有权或共享所有权，如图7-13所示  
![7-13](../syn_pic/the_datawarehouse_toolkit/7-13.png)
<center>图7-13 包含共享所有权的不规则层次桥接表 5</center>

### 7.4.5 随时间变化的不规则层次  
包含另外两个日期/时间戳的不规则层次桥接表可方便缓慢地变化层次，如图7-14所示  
![7-14](../syn_pic/the_datawarehouse_toolkit/7-14.png)
<center>图7-14 随时间变化的不规则层次的桥接表</center>

警告：在使用类似图7-14所示的桥接表时，查询必须被约束为单一日期/时间，以“冻结”桥接表，保持层次的单一一致性视图  
### 7.4.6 修改不规则层次
组织映射桥接表可以被方便地修改。假设希望将节点4、5、6从其原始位置（向上为节点2）移动到一个新位置（向上为节点9），如图7-15所示  
![7-15](../syn_pic/the_datawarehouse_toolkit/7-15.png)
<center>图7-15 改变图7-8所示的组织结构 5</center>

在此静态情况下，桥接表仅仅反映了当前上卷结构，仅仅需呀删除树中指向节点4、5和6包含的组的高层路径  
在随时间变化的情况下，桥接表包含一对日期/时间戳，逻辑是类似的  
上述改变桥接表的简单方法避免了在改变其他类型层次模型时出现的噩梦般的情形  
### 7.4.7 其他不规则层次的建模方法
除了在组织维度中使用递归指针外，至少存在两种建模不规则层次的方法。这两种方法都涉及设置在组织维度中的聪明列  
一种模式是在组织维度表上增加路径字符属性，如图7-16所示。通过使用通配符约束路径字符，可以实现针对树的查询，例如： 5  
+ A\*检索整棵树，其中星号表示可变长度通配符  
+ \*.仅检索左节点  
+ ?+检索顶端节点，问号是单个字符通配符  

![7-16](../syn_pic/the_datawarehouse_toolkit/7-16.png)
<center>图7-16 其他使用路径字符属性的不规则层次设计</center>

路径字符方法对由于组织发生变化而带来的重新标记非常敏感，如果在树中插入新节点，则处于该节点的同一父节点之下的所有右侧节点都必须重新标记 5  
另外一种类似的模式是计算机科学家经常使用的改进的前序树遍历方法，如图7-17所示的号码，每个节点包含一对号码用于标识该节点之下的所有节点  
![7-17](../syn_pic/the_datawarehouse_toolkit/7-17.png)
<center>图7-17 使用改进的前序树遍历方法的其他不规则层次设计</center>

### 7.4.8 应用于不规则层次的桥接表方法的优点  
虽然桥接表增加了更多的ETL设置工作，并且增加了查询的负担，但它为分析无法确定深度的不规则层次提供了良好的灵活性。特别是，桥接表允许：  
+ 在查询时选座其他上卷结构
+ 共享所有权上卷 5
+ 随时间变化的不规则层次  
+ 当节点早于缓慢变化维度类型2变化时影响较小
+ 结构改变时影响较小  

可以使用组织的层次桥接表跨预算链上所有三个事实表获取事实。图7-18给出了组织映射表如何与是哪个预算链事实表连接  
![7-18](../syn_pic/the_datawarehouse_toolkit/7-18.png)
<center>图7-18 横向钻取和上卷预算链 5</center>

## 7.5 合并事实表
在最后这节中，我们将对由不同业务过程采用横向钻取事实表建立的度量进行比较，例如，预算和承诺  
通常，业务管理人员喜欢比较实际与预算的差异。图7-19显示了实际与预算额，以及通过公共维度获得的差异（计算获得的差异）  
![7-19](../syn_pic/the_datawarehouse_toolkit/7-19.png)
<center>图7-19 实际与预算合并事实表</center>

再次考虑跨国组织情况，基于有效的转换比率，会看到本地和等价的标准货币的实际金额  
以共同粒度从多个组织业务过程合并度量的事实表被称为合并事实表 5  
注意：来自多个业务过程的事实合并到合并事实表时，它们必须具有同样等级的粒度 5  
## 7.6 OLAP角色及分析方案包
在关系数据库环境下讨论财务维度模型时，值得注意的是多维OLAP提供商长久以来在该领域中扮演的角色  
OLAP适合处理复杂的组织上卷，以及复杂的计算工作，包括行间操作  
按照总账处理的标准属性，购买总账包而不是试图从头开始建立是近年来比较流行的路线 5  
## 7.7 本章小结
本章主要关注财务总账数据，包括周期性快照以及日记账条目事务  
本章使用为人熟知的组织上卷结构展示了如何建模可变深度的复杂不规则层次  
本章探讨了预算过程链的事件系列  